In [3]:
import dash_bootstrap_components as dbc
from dash import Input, Output, dcc, html
import dash
import warnings 
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd
from shapely import wkb, speedups
import geopandas as gpd
speedups.disable()
import plotly.express as px
import plotly.graph_objects as go
import pymssql
from joblib import load


database = 'Pushing-P-DB'
table2 = 'dbo.NJ_Census_Tracts'
table3 = 'dbo.NJ_County_Info'
table = 'dbo.Live_Feed'
user = 'pushing_p'
password  = 't3stP@ssword'
server = 'gen10-data-fundamentals-22-02-sql-server.database.windows.net'

col_dict = {
'NUI, Under 6 years': '% Uninsured, Under 6 years',
'NUI, 6-18 years': '% Uninsured, 6-18 years',
'NUI, 19 to 25 years': '% Uninsured, 19 to 25 years',
'NUI, 26 to 34 years': '% Uninsured, 26 to 34 years',
'NUI, 35 to 44 years': '% Uninsured, 35 to 44 years',
'NUI, 45 to 54 years': '% Uninsured, 45 to 54 years',
'NUI, 55 to 64 years': '% Uninsured, 55 to 64 years',
'NUI, 65 years and older': '% Uninsured, 65 years and older',
'NUI, Men': '% Uninsured, Men',
'NUI, Women': '% Uninsured, Women',
'NUI, White': '% Uninsured, White',
'NUI, African American': '% Uninsured, African American',
'NUI, American Indian & Alaska Natives': '% Uninsured, American Indian & Alaska Natives',
'NUI, Asians': '% Uninsured, Asians',
'NUI, Native Hawaiians & Pacific Islanders': '% Uninsured, Native Hawaiians & Pacific Islanders',
'NUI, Other Races': '% Uninsured, Other Races'}

percent_cols = list(col_dict.values())
numeric_cols = list(col_dict.keys())


def Get_Data(table):
    try:
        conn = pymssql.connect(server,user,password,database)

        # Query select all rows from SQL tables to insert into their respective DataFrames
        query = f'SELECT * FROM {table}'

        df = pd.read_sql(query, conn)

        def convert_to_polygon(hex):
            return wkb.loads(hex, hex=True)

        df['geometry'] = df['geometry'].apply(convert_to_polygon)
        
        df.dropna(inplace=True)

        df[['City Population','Uninsured Population']] = df[['City Population','Uninsured Population']].astype(int)
        
        df = pd.DataFrame(df)
        df['% Uninsured'] = (df['Uninsured Population']/df['City Population'])
        df['% Uninsured'] = df['% Uninsured'].round(decimals = 5)
        df['Insured Population'] = df['City Population'] - df['Uninsured Population']
        df['Insured Population'] = df['Insured Population'].round(decimals = 5)
        df['Rate Of Insured'] = df['Insured Population'] / df['City Population']
        df['Rate Of Insured'] = df['Rate Of Insured'].round(decimals = 5)

        df[numeric_cols] = df[numeric_cols].astype(float)

        df[percent_cols] = df[numeric_cols].div(df['Uninsured Population'], axis=0)

        df[percent_cols] = df[percent_cols].apply(lambda x: x * 100)

        # converting to geopandas dataframe
        df = gpd.GeoDataFrame(df, geometry=df['geometry'], crs = 'epsg:4269')

        # This is where I format the TimeStamp to "ms" and then I have to for some reason use ".dt.strftime('%Y-%m-%d %H:%M:%S')" because the format in the table chances when I print it out
#         df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')

        # This is where I order based off of timestamp for my fifth dataframe
#         df = df.sort_values(by='TimeStamp',ascending=False)

        return df

    except Exception as e:
        raise e

df = Get_Data(table) #Live_Feed table
tracts = Get_Data(table2)
counties = Get_Data(table3)
results = load('areal interpolation joblib.model')

def Make_Plots(df):
    
    county_name = df.County[0]

    # Here I set the colors I will be using for the visualizations
    colors = {
        'background': '#111111',
        'text': '#7FDBFF'
    }
#  1. GAUGE VISUAL PLACE
    df1 = df[['Uninsured Population','City Population']].sum()
    # df1

    fig_1 = go.Figure(go.Indicator(
        mode = "gauge+number+delta",
        value = df1[['Uninsured Population']].sum(),
        domain = {'x': [0, 1], 'y': [0, 1]},
        title = {'text': f'Total Number of Uninsured for {county_name}', 'font': {'size': 15.5}},
        delta = {'reference': df1['City Population'], 'increasing': {'color': "RebeccaPurple"}},
        gauge = {
            'axis': {'range': [None, 1000000], 'tickwidth': 1, 'tickcolor': "red"},
            'bar': {'color': "darkblue"},
            'bgcolor': "red",
            'borderwidth': 2,
            'bordercolor': "gray",
            'steps': [
                {'range': [0, 500000], 'color': 'cyan'},
                {'range': [500000, 750000], 'color': 'yellow'}],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 1000000}}))

    fig_1.update_layout(paper_bgcolor = "#111111", font = {'color': "lightblue", 'family': "Arial"}, width = 375)

    
# 4. BAR % UNINSURED VS. AGE
   
    df4= df[['State','County','City','NUI, Under 6 years', 'NUI, 6-18 years', 'NUI, 19 to 25 years',
    'NUI, 26 to 34 years', 'NUI, 35 to 44 years', 'NUI, 45 to 54 years',
    'NUI, 55 to 64 years', 'NUI, 65 years and older','% Uninsured']].copy()

    df4 = df4 [['NUI, Under 6 years', 'NUI, 6-18 years', 'NUI, 19 to 25 years',
    'NUI, 26 to 34 years', 'NUI, 35 to 44 years', 'NUI, 45 to 54 years',
    'NUI, 55 to 64 years', 'NUI, 65 years and older']].sum()

    df4 = pd.DataFrame(df4)

    df4 = df4.reset_index()

    # Didnt use an equals sign because this just renames the existing dataframe instead of renaming a copy of the dataframe.
    df4.rename(columns = {'index':'Age', 0:'Number Of Uninsured'}, inplace = True)

    State_Uninsured = df['Uninsured Population'].sum()

    df4['Uninsured Rate By Age'] = df4['Number Of Uninsured']/State_Uninsured * 100

    df4['Uninsured Rate By Age'] = df4['Uninsured Rate By Age'].round(decimals = 2)


    fig_4 = px.bar(df4, x='Age', y='Uninsured Rate By Age',color_discrete_sequence=["green"], title= f'The Rate Of Uninsured By Age For {county_name}', height = 500, width = 700)
    fig_4.update_layout(xaxis={'categoryorder':'total descending'}, width = 625)


    # this is where I center the title for the visualization
    fig_4.update_layout(
        title=dict(x=0.5), #set title in the center
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
    )
   
    # 5. BAR % UNINSURED VS. RACE
   
    df5= df[['NUI, White', 'NUI, African American','NUI, American Indian & Alaska Natives',
    'NUI, Asians','NUI, Native Hawaiians & Pacific Islanders', 'NUI, Other Races','% Uninsured']].copy()

    df5.rename(columns = {'NUI, American Indian & Alaska Natives':'NUI, American I. & Alaska N.', 'NUI, Native Hawaiians & Pacific Islanders':'NUI, Native H. & Pacific I.'}, inplace = True)

    df5 = df5[['NUI, White', 'NUI, African American','NUI, American I. & Alaska N.',
    'NUI, Asians','NUI, Native H. & Pacific I.', 'NUI, Other Races']].sum()

    df5 = pd.DataFrame(df5)

    df5 = df5.reset_index()

    # # Didnt use an equals sign because this just renames the existing dataframe instead of renaming a copy of the dataframe.
    df5.rename(columns = {'index':'Race', 0:'Number Of Uninsured'}, inplace = True)

    State_Uninsured = df['Uninsured Population'].sum()

    df5['Uninsured Rate By Race'] = df5['Number Of Uninsured']/State_Uninsured * 100

    df5['Uninsured Rate By Race'] = df5['Uninsured Rate By Race'].round(decimals = 2)


    fig_5 = px.bar(df5, x='Race', y='Uninsured Rate By Race',color_discrete_sequence=["blue"], title= f'The Rate Of Uninsured By Race For {county_name}', height = 500, width = 700)
    fig_5.update_layout(xaxis={'categoryorder':'total descending'})

    # this is where I center the title for the visualization
    fig_5.update_layout(
        title=dict(x=0.45), #set title in the center
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        width = 515,
        title_font=dict(size= 15)
        
    )


# 6. TABLE TOP 10 UNINSURED CITIES VISUAL PLACE

    df6= df[['City','Uninsured Population','% Uninsured']].copy()

    df6 = df6.sort_values(by='% Uninsured', ascending =False)

    df6 = df6.head(10)

    fig_6 = go.Figure(data=[go.Table(
        header=dict(values=df6.columns,
                    fill_color='#3F7674',
                    align='left'),
        cells=dict(values=[df6.City, df6['% Uninsured'], df6['Uninsured Population']],
                fill_color='black',
                align='left'))
    ])

    fig_6.update_layout(
        title = f'Top 10 Uninsured Cities by Rate in {county_name}', # this is where I add the title for my table
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color='#7FDBFF'
    )

    fig_6.update_layout(
        title=dict(x=0.5)
    )

# 7. DONUTS VISUAL ONE PLACE
    df7 = df.copy()

    df7['Employment Rate'] = 100 - df['Unemployment Rate (16 & Over)']

    df7.rename(columns = {'Unemployment Rate (16 & Over)': 'Unemployment Rate'}, inplace = True)

    df7 = df7[['County','Insured Population', 'Uninsured Population','Unemployment Rate','Employment Rate']].sum()

    df7 = pd.DataFrame(df7)

    df7 = df7.reset_index()

    df7.rename(columns = {'index':'Variables', 0:'Sum'}, inplace = True)

    df7 = df7.loc[1:]

    df7_1 = df7.iloc[:2,:].copy()

    colors = ['lime','orange']
    fig_7 = go.Figure(data = go.Pie(values = df7_1.Sum, 
                            labels = df7_1.Variables, hole = 0.6,
                            marker_colors = colors,
                    ))
    fig_7.update_traces(
                    title_font=dict(size=30,family='Verdana', 
                                    color='white'),
                                    hoverinfo='label+percent',
                                    textinfo='percent', 
                                    textfont_size=20,
                    )

    fig_7.update_layout(legend=dict(y=1.1, x = 0.8), title = f'Insured and Uninsured Population in {county_name}',
    paper_bgcolor = '#111111', font = {'color': "#7FDBFF", 'family': "Arial"})
    fig_7.update_layout(title=dict(x=0.5), title_font=dict(size= 15)) #set title in the center)

# 8. DONUTS VISUAL TWO PLACE

    df7_2 = df7.iloc[2:4,:].copy()

    colors = ['red','blue']
    fig_8 = go.Figure(data = go.Pie(values = df7_2.Sum, 
                            labels = df7_2.Variables, hole = 0.6,
                            marker_colors = colors,
                    ))
    fig_8.update_traces(
                    title_font=dict(size=30,family='Verdana', 
                                    color='white'),
                                    hoverinfo='label+percent',
                                    textinfo='percent', 
                                    textfont_size=20,
                    )

    fig_8.update_layout(legend=dict(y=1.1, x = 0.8), title = f'Employed and Unemployed Rate in {county_name}',
    paper_bgcolor = "#111111", font = {'color': "#7FDBFF", 'family': "Arial"})
    fig_8.update_layout(title=dict(x=0.5), title_font=dict(size= 15)) #set title in the center)

# 9. HEAT MAP % OF UNINSURED VISUAL PLACE




# 10. ML PREDICTION VISUAL




    return fig_1, fig_4, fig_5, fig_6, fig_7, fig_8  

fig_1, fig_4, fig_5, fig_6, fig_7, fig_8 = Make_Plots(df)

# Set default variable for maps
default_variable = '% Uninsured'

def Make_Plots_Maps(counties, cities, tracts, variable):

    county_name = cities.at[0, 'County']
    counties = counties.loc[counties['County'] == county_name]
    counties = counties.set_index('County')

    # County Map
    px.set_mapbox_access_token('pk.eyJ1IjoiYWhhZGg3NjIiLCJhIjoiY2wzaTBqbnQ2MGU2cjNqbzZpNXFiZHk1eSJ9.UTxbELXv9gk6QiowY1VgqA')
    fig_9 = px.choropleth_mapbox(counties, geojson=counties.__geo_interface__, locations=counties.index, color= variable,
                            mapbox_style="outdoors", color_continuous_scale='Viridis_r',
                            zoom=6, center = {"lat": 40.058300, "lon": -74.405700},  # Lat long is centerpoint of NJ State
                            )

    fig_9.update_layout(
        margin=dict(l=35, r=35, t=35, b=35), # change margin dimensions
        title = f'{variable} in {county_name}',
        # width = 380,
        # height = 380,
        paper_bgcolor="#111111", # set background color
        font = {'color': "#7FDBFF", 'family': "Arial"},
        coloraxis_showscale=False, # hides colorscale
        title_x = 0.5 # centers title
    )

    fig_9['layout']['title']['font'] = dict(size=14) # Set title fontsize

    fig_9.update_geos(fitbounds="locations")

    # Get Centerpoint of County
    lon = float(counties.centroid.x.values[0])
    lat = float(counties.centroid.y.values[0])

    cities = cities.set_index('City')

    # City Map
    fig_10 = px.choropleth_mapbox(cities, geojson=cities.__geo_interface__, locations=cities.index, color = variable,
                            mapbox_style="outdoors", color_continuous_scale='Viridis_r',
                            zoom=7.0, center = {"lat": lat , "lon": lon},
                            )

    fig_10.update_layout(
        margin=dict(l=35, r=35, t=35, b=35),
        title = f'{variable} in {county_name} Cities',
        # width = 500,
        # height = 380,
        paper_bgcolor="#111111",
        font = {'color': "#7FDBFF", 'family': "Arial"},
        title_x = 0.375
        )

    fig_10['layout']['title']['font'] = dict(size=14)
    fig_10.update_geos(fitbounds="locations")

    # Predicted Census Tracts Map (Machine Learning Model)
    model = results.loc[results['County'] == county_name]
    model = model.set_index('Census Tract')

#     extensive_variables = numeric_cols + ['Uninsured Population']
#     model = area_interpolate(source_df=cities, target_df=tracts, intensive_variables = ['Unemployment Rate (16 & Over)'], extensive_variables = extensive_variables)

#     model = load('areal interpolation joblib.model') # Creates a dataframe with the results of the Machine Learning Model
 

    fig_11 = px.choropleth_mapbox(model, geojson=model.__geo_interface__, locations=model.index, color = variable,
                            mapbox_style="outdoors", color_continuous_scale='Viridis_r',
                            zoom=7.8, center = {"lat": lat , "lon": lon},
                            )

    fig_11.update_layout(
        margin=dict(l=35, r=35, t=35, b=35),
        title = f'{variable} in {county_name} Census Tracts (Predicted)',  title_x = 0.40,
        # width = 380,
        # height = 380,
        paper_bgcolor="#111111",
        font = {'color': "#7FDBFF", 'family': "Arial"})

    fig_11['layout']['title']['font'] = dict(size=14)
    fig_11.update_geos(fitbounds="locations")

     # Predicted Census Tracts Map
    tracts = tracts.loc[tracts['County'] == county_name]
    tracts = tracts.set_index('Census Tract')

    # Actual Census Tracts Map
    fig_12 = px.choropleth_mapbox(tracts, geojson=tracts.__geo_interface__, locations=tracts.index, color = variable,
                            mapbox_style="outdoors", color_continuous_scale='Viridis_r',
                            zoom=7.8, center = {"lat": lat , "lon": lon},
                            )

    fig_12.update_layout(
        margin=dict(l=35, r=35, t=35, b=35),
        title = f'{variable} in {county_name} Census Tracts (Actual)',  title_x = 0.40,
        # width = 380,
        # height = 380,
        paper_bgcolor="#111111",
        font = {'color': "#7FDBFF", 'family': "Arial"})

    fig_12['layout']['title']['font'] = dict(size=14)
    fig_12.update_geos(fitbounds="locations")


    return fig_9, fig_10, fig_11, fig_12

fig_9, fig_10, fig_11, fig_12 = Make_Plots_Maps(counties, df, tracts, default_variable)

# CSS Styling

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#545853",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4", style = {'color':'#7FDBFF'}),
        html.Hr(),
        html.P(
            "A simple sidebar layout with navigation links", className="lead", style = {'color':'#7FDBFF'}
        ),
        dbc.Nav(
            [
                dbc.NavLink("Page 1", href="/page-1", active=""),
                dbc.NavLink("Page 2", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", style=CONTENT_STYLE)

app.layout = html.Div([dcc.Location(id="url"), sidebar, content], style={'backgroundColor':'#111111'})


@app.callback(Output("page-content", "children"), [Input("url", "pathname")])
def render_page_content(pathname):
    if pathname == "/page-1" or pathname=="/":
        
        return[
        html.Div([

#             app.layout = html.Div(
#                 style={'backgroundColor':'#111111'}),    

            # the four graphs
               
                
                    html.Div(
                    children = dcc.Graph(
                        id = 'Gauge',
                        figure = fig_1,
                      #  config={"displayModeBar": False},
                    ),
                    style={'width': '40%', 'display': 'inline-block'},
                ),

                    html.Div(
                    children = dcc.Graph(
                        id = '% UNINSURED VS AGE',
                        figure = fig_4,
                        #config={"displayModeBar": False},
                    ),
                    style={'width': '40%', 'display': 'inline-block'},    
                ),

                    html.Div(
                    children = dcc.Graph(
                        id = 'Race',
                        figure = fig_5,
                        # config={"displayModeBar": False},
                    ),
                    style={'width': '50%','display': 'inline-block'}, # height part of style doesn't work
                ),


                     html.Div(
                    children = dcc.Graph(
                        id = 'Table',
                        figure = fig_6,
                        #config={"displayModeBar": False},
                    ),
                    style={'width': '50%', 'display': 'inline-block'},
                ),



                    html.Div(
                    children = dcc.Graph(
                        id = 'donut-1',
                        figure = fig_7,
                        #config={"displayModeBar": False},
                    ),
                    style={'width': '50%', 'display': 'inline-block'},
                ),



                    html.Div(
                    children = dcc.Graph(
                        id = 'donut-2',
                        figure = fig_8,
                        #config={"displayModeBar": False},
                    ),
                    style={'width': '50%', 'display': 'inline-block'},
                ),

                    dcc.Interval(
                    id='interval-component',
                    interval= 2000, # in milliseconds (there will be an update once every minute)
                    n_intervals=0)],       
            className = 'double-graph'
        )
        ]        
        
                     
            
    elif pathname == "/page-2":

        return [html.Div([

            # Dash Application Layout
            html.Div([
                dcc.Dropdown(['% Uninsured'] +percent_cols, '% Uninsured', id='dropdown',
                clearable = False,
                searchable = False),
                html.Div(id='dd-output-container')
            ]),
            
            dcc.Graph(id = 'County Map',
            figure = fig_9, style={'width': '40%', 'display': 'inline-block'}), 

            dcc.Graph(id = 'City Map',
            figure = fig_10, style={'width': '60%', 'display': 'inline-block'}),

            dcc.Graph(id = 'Predicted Census Tract Map',
            figure = fig_11, style={'width': '50%', 'display': 'inline-block'}),

            dcc.Graph(id = 'Actual Census Tract Map',
            figure = fig_12, style={'width': '50%', 'display': 'inline-block'}),

            dcc.Interval(
            id='interval-component2',
            interval= 2000, # in milliseconds (there will be an update once every minute)
            n_intervals=0)])]

        



@app.callback(
Output("Gauge", "figure"),
Output("% UNINSURED VS AGE", "figure"),
Output("Race", "figure"),
Output("Table", "figure"),
Output("donut-1", "figure"),
Output("donut-2", "figure"),
Input('interval-component', "n_intervals")# this input is the dcc.Interval id ("interval-component") and "n_intervals" 
)

    
def update_charts(n):
    df = Get_Data(table)
    fig_1, fig_4, fig_5, fig_6, fig_7, fig_8  = Make_Plots(df)
    return fig_1, fig_4, fig_5, fig_6, fig_7, fig_8
    
@app.callback(
Output("County Map", "figure"),
Output("City Map", "figure"),
Output("Predicted Census Tract Map", "figure"),
Output("Actual Census Tract Map", "figure"),
Input('interval-component2', "n_intervals"),# this input is the dcc.Interval id ("interval-component") and "n_intervals"
Input("dropdown", "value")
)

def update_charts2(n, dropdown):

    df = Get_Data(table)
    fig_9,fig_10,fig_11,fig_12 = Make_Plots_Maps(counties, df, tracts, dropdown)

    return fig_9, fig_10, fig_11, fig_12


if __name__ == '__main__':
    app.run_server(debug = False)

KeyError: "['City Population'] not in index"

In [4]:
df.columns

Index(['City', 'City Population', 'County', 'Median Household Income',
       'NUI, 19 to 25 years', 'NUI, 26 to 34 years', 'NUI, 35 to 44 years',
       'NUI, 45 to 54 years', 'NUI, 55 to 64 years', 'NUI, 6-18 years',
       'NUI, 65 years and older', 'NUI, African American',
       'NUI, American Indian & Alaska Natives', 'NUI, Asians', 'NUI, Men',
       'NUI, Native Hawaiians & Pacific Islanders', 'NUI, Other Races',
       'NUI, Under 6 years', 'NUI, White', 'NUI, Women', 'State', 'Timestamp',
       'Unemployment Rate (16 & Over)', 'Uninsured Population', 'geometry',
       '% Uninsured', 'Insured Population', 'Rate Of Insured',
       '% Uninsured, Under 6 years', '% Uninsured, 6-18 years',
       '% Uninsured, 19 to 25 years', '% Uninsured, 26 to 34 years',
       '% Uninsured, 35 to 44 years', '% Uninsured, 45 to 54 years',
       '% Uninsured, 55 to 64 years', '% Uninsured, 65 years and older',
       '% Uninsured, Men', '% Uninsured, Women', '% Uninsured, White',
       '% U